In [1]:
import warnings
warnings.filterwarnings('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 5

https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/Prompt_Tuning_PEFT.ipynb

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True)


In [3]:
def get_outputs(model, inputs, max_length=100):
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        repetition_penalty=1.5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    print(outputs)  # Add this line
    return outputs


In [4]:
input_prompt = tokenizer(" I want you to act as a motivatinal coach for me", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model, input_prompt, max_length=50)
print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))


tensor([[  473,  4026,  1152,   427,  1769,   661,   267, 22675,   278,  4001,
         67604,   613,  1074,     2]])
[' I want you to act as a motivatinal coach for me']


In [5]:
dataset_prompt = "fka/awesome-chatgpt-prompts"
data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda samples:tokenizer(samples["prompt"]), batched=True)
train_sample_prompt = data_prompt["train"].select(range(50))


In [6]:
for i, sample in enumerate(train_sample_prompt['prompt'][:5]):
    print(f'Quote {i+1}: {sample} \n\n\n')

Quote 1: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd 



Quote 2: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. My first sentence is "is

In [8]:
import random

samples = random.sample(train_sample_prompt['prompt'], 5)
for i, sample in enumerate(samples):
    print(f'Quote {i+1}: {sample} \n\n\n')

Quote 1: Need somebody with expertise on automobiles regarding troubleshooting solutions like; diagnosing problems/errors present both visually & within engine parts in order to figure out what's causing them (like lack of oil or power issues) & suggest required replacements while recording down details such fuel consumption type etc., First inquiry – “Car won't start although battery is full charged” 



Quote 2: I want you to act as an accountant and come up with creative ways to manage finances. You'll need to consider budgeting, investment strategies and risk management when creating a financial plan for your client. In some cases, you may also need to provide advice on taxation laws and regulations in order to help them maximize their profits. My first suggestion request is “Create a financial plan for a small business that focuses on cost savings and long-term investments". 



Quote 3: I want you to act as a real estate agent. I will provide you with details on an individual loo

In [9]:
dataset_sentences = load_dataset("Abirate/english_quotes")
data_sentences = dataset_sentences.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample_sentences = data_sentences["train"].select(range(25))
train_sample_sentences = train_sample_sentences.remove_columns(['author', 'tags'])


In [12]:
import random

samples = random.sample(train_sample_prompt['prompt'], 5)
for i, sample in enumerate(samples):
    print(f'Quote {i+1}: {sample} \n\n\n')

Quote 1: I want you to act as a philosophy teacher. I will provide some topics related to the study of philosophy, and it will be your job to explain these concepts in an easy-to-understand manner. This could include providing examples, posing questions or breaking down complex ideas into smaller pieces that are easier to comprehend. My first request is "I need help understanding how different philosophical theories can be applied in everyday life." 



Quote 2: I want you to act as a career counselor. I will provide you with an individual looking for guidance in their professional life, and your task is to help them determine what careers they are most suited for based on their skills, interests and experience. You should also conduct research into the various options available, explain the job market trends in different industries and advice on which qualifications would be beneficial for pursuing particular fields. My first request is "I want to advise someone who wants to pursue a 

In [9]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generation_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  #The added virtual tokens are initializad with random numbers
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [10]:
peft_model_prompt = get_peft_model(foundational_model, generation_config)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [11]:


peft_model_sentences = get_peft_model(foundational_model, generation_config)
print(peft_model_sentences.print_trainable_parameters())



trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [12]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=True, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        num_train_epochs=epochs
    )
    return training_args

In [13]:
import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")
output_directory_sentences = os.path.join(working_dir, "peft_outputs_sentences")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)
if not os.path.exists(output_directory_sentences):
    os.mkdir(output_directory_sentences)

In [14]:
training_args_prompt = create_training_arguments(output_directory_prompt, 0.003, NUM_EPOCHS)
training_args_sentences = create_training_arguments(output_directory_sentences, 0.003, NUM_EPOCHS)

In [15]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, bloomz-560M
        args=training_args, #The args for the training.
        train_dataset=train_dataset, #The dataset used to tyrain the model.
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
    )
    return trainer

In [16]:
#Training first model.
trainer_prompt = create_trainer(peft_model_prompt, training_args_prompt, train_sample_prompt)
trainer_prompt.train()

  0%|          | 0/35 [00:00<?, ?it/s]

{'train_runtime': 177.8568, 'train_samples_per_second': 1.406, 'train_steps_per_second': 0.197, 'train_loss': 3.592269025530134, 'epoch': 5.0}


TrainOutput(global_step=35, training_loss=3.592269025530134, metrics={'train_runtime': 177.8568, 'train_samples_per_second': 1.406, 'train_steps_per_second': 0.197, 'train_loss': 3.592269025530134, 'epoch': 5.0})

In [17]:
#Training second model.
trainer_sentences = create_trainer(peft_model_sentences, training_args_sentences, train_sample_sentences)
trainer_sentences.train()

  0%|          | 0/20 [00:00<?, ?it/s]

{'train_runtime': 53.3114, 'train_samples_per_second': 2.345, 'train_steps_per_second': 0.375, 'train_loss': 4.46654052734375, 'epoch': 5.0}


TrainOutput(global_step=20, training_loss=4.46654052734375, metrics={'train_runtime': 53.3114, 'train_samples_per_second': 2.345, 'train_steps_per_second': 0.375, 'train_loss': 4.46654052734375, 'epoch': 5.0})

In [23]:
trainer_prompt.model.save_pretrained(output_directory_prompt)
trainer_sentences.model.save_pretrained(output_directory_sentences)

In [27]:
from peft import PeftModel

loaded_model_prompt = PeftModel.from_pretrained(foundational_model,
                                         output_directory_prompt,
                                         #device_map='auto',
                                         is_trainable=False)

In [28]:
loaded_model_prompt_outputs = get_outputs(loaded_model_prompt, input_prompt)
print(tokenizer.batch_decode(loaded_model_prompt_outputs, skip_special_tokens=True))

tensor([[  473,  4026,  1152,   427,  1769,   661,   267, 22675,   278,  4001,
         67604,   613,  1074,     2]])
[' I want you to act as a motivatinal coach for me']
